In [26]:
import statsapi
import psycopg2
from baseball_data import generateGamePksFromDates
import psycopg2.extras
import time

In [21]:
conn = psycopg2.connect(
    host = "localhost",
    database = "baseball",
    user = "karisch",
    password = "cocacola",
    port = 5432
)
#conn.autocommit = True

In [28]:
conn.commit()
cur.close()
conn.close()

In [3]:
def populateVenueTable(dbConnection):
    teams = statsapi.get('teams', {})

    venueDict = {'venueIds': [], 'venueNames': []}
    for team in teams['teams']:
        activeVenue = team['venue']['id']
        if activeVenue not in venueDict['venueIds']:
            venueDict['venueIds'].append(team['venue']['id'])
            venueDict['venueNames'].append(team['venue']['name'])

    with dbConnection.cursor() as cur:
        for i in range(len(venueDict['venueIds'])):
            cur.execute(
                """
                INSERT INTO venues(id, name)
                    VALUES (%s, %s)
                """,
                (venueDict['venueIds'][i], venueDict['venueNames'][i])
                )

In [4]:
populateVenueTable(conn)

In [18]:
teams = statsapi.get('teams', {})
venueDict = {'venueIds': [], 'venueNames': []}
manyList = []
for team in teams['teams']:
    activeVenue = team['venue']['id']
    if activeVenue not in venueDict['venueIds']:
        venueDict['venueIds'].append(team['venue']['id'])
        venueDict['venueNames'].append(team['venue']['name'])
        manyList.append((team['venue']['id'], team['venue']['name']))

In [22]:
cur = conn.cursor()

In [19]:
cur.executemany(
    """
    INSERT INTO venues(id, name)
        VALUES (%s, %s)
    """,
    manyList
)

In [27]:
psycopg2.extras.execute_batch(
    cur,
    """
    INSERT INTO venues(id, name)
        VALUES (%s, %s)
    """,
    manyList
)